In [ ]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash


--2021-12-24 14:42:53--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://setup.johnsnowlabs.com/colab.sh [following]
--2021-12-24 14:42:53--  https://setup.johnsnowlabs.com/colab.sh
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2021-12-24 14:42:54--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:44

In [ ]:
!wget http://setup.johnsnowlabs.com/colab.sh 

! cat /content/colab.sh

--2021-12-24 14:43:54--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://setup.johnsnowlabs.com/colab.sh [following]
--2021-12-24 14:43:55--  https://setup.johnsnowlabs.com/colab.sh
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2021-12-24 14:43:55--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:44

In [ ]:
import pandas as pd
import numpy as np

# 2.1 spark related
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import DoubleType, StringType,StructField,StructType
#Replace part of string with another string
from pyspark.sql.functions import regexp_replace

# 2.2 Spark-nlp related
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *

In [ ]:
# 2.3 Create Spark session
#     And start sparknlp

spark = sparknlp.start()

In [ ]:
# 2.4 Show multiple command outputs from a cell

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
from sparknlp.pretrained import PretrainedPipeline
pipeline = PretrainedPipeline('analyze_sentiment', lang = 'en')
result =  pipeline.annotate("Movie was worthless ")
print(result)

analyze_sentiment download started this may take some time.
Approx size to download 4.9 MB
[OK!]
{'checked': ['Movie', 'was', 'worthless'], 'document': ['Movie was worthless '], 'sentiment': ['negative'], 'token': ['Movie', 'was', 'worthless'], 'sentence': ['Movie was worthless']}


In [ ]:
# 2.3 Mount gdrive to read data
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
# 4.0 Read data directly in spark a normal manner:

data = spark.read.csv(
                      path = '/gdrive/MyDrive/IMDB Dataset.csv',
                      inferSchema=True,
                      header=True
                      )

In [ ]:
data.show(4)
data.count() 

+--------------------+--------------------+
|              review|           sentiment|
+--------------------+--------------------+
|One of the other ...|            positive|
|"A wonderful litt...| not only is it w...|
|"I thought this w...| but spirited you...|
|Basically there's...|            negative|
+--------------------+--------------------+
only showing top 4 rows



50000

In [ ]:
data.head()

Row(review="One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is du

In [ ]:
# 5.0 First define a Spark schema:
schema = StructType([ \
                     StructField("review",StringType(),True), \
                     StructField("sentiment",StringType(),True), \
  ])
 

In [ ]:
# 5.1 Next read the data:

trainDataset = spark.read  \
                 .option("quote", "\"") \
                 .option('escape', "\"") \
                 .option("multiLine", "true")  \
                 .option("schema" , schema)  \
                 .option("header", "true") \
                 .csv('/gdrive/MyDrive/IMDB Dataset.csv')

In [ ]:
# 5.2 The result shows complete match with pandas:
trainDataset.show()
trainDataset.count() 

+--------------------+---------+
|              review|sentiment|
+--------------------+---------+
|One of the other ...| positive|
|A wonderful littl...| positive|
|I thought this wa...| positive|
|Basically there's...| negative|
|Petter Mattei's "...| positive|
|Probably my all-t...| positive|
|I sure would like...| positive|
|This show was an ...| negative|
|Encouraged by the...| negative|
|If you like origi...| positive|
|Phil the Alien is...| negative|
|I saw this movie ...| negative|
|So im not a big f...| negative|
|The cast played S...| negative|
|This a fantastic ...| positive|
|Kind of drawn in ...| negative|
|Some films just s...| positive|
|This movie made i...| negative|
|I remember this f...| positive|
|An awful film! It...| negative|
+--------------------+---------+
only showing top 20 rows



50000

In [ ]:
# 5.3 Distribution of classes:
from pyspark.sql.functions import col

# 5.3.1
trainDataset.groupBy("sentiment") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+---------+-----+
|sentiment|count|
+---------+-----+
| positive|25000|
| negative|25000|
+---------+-----+



In [ ]:
(trainingData, testData) = trainDataset.randomSplit([0.8, 0.2], seed = 100)

# 6.0.1
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 40055
Test Dataset Count: 9945


In [ ]:
#pipelinig for data cleaning
# 6.1
document_assembler = DocumentAssembler() \
    .setInputCol("review") \
    .setOutputCol("document")

# 6.2    
tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

# 6.3      
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

# 6.4
stopwords_cleaner = StopWordsCleaner()\
    .setInputCols("normalized")\
    .setOutputCol("cleanTokens")\
    .setCaseSensitive(False)

# 6.5
lemma = LemmatizerModel.pretrained('lemma_antbnc') \
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("lemma")

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


In [ ]:
# 7.0
glove_embeddings = WordEmbeddingsModel().pretrained() \
      .setInputCols(["document",'lemma'])\
      .setOutputCol("embeddings")\
      .setCaseSensitive(False)

# 7.1
embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["document", "embeddings"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")

# 7.2
classsifierdl = ClassifierDLApproach()\
      .setInputCols(["sentence_embeddings"])\
      .setOutputCol("class")\
      .setLabelColumn("sentiment")\
      .setMaxEpochs(100)\
      .setEnableOutputLogs(True)
      #.setOutputLogsPath('logs')

# 7.3
clf_pipeline = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            lemma, 
            glove_embeddings,
            embeddingsSentence,
            classsifierdl])

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


In [ ]:
# 8.0 Train 
%%time

clf_pipelineModel = clf_pipeline.fit(trainingData)

CPU times: user 7.87 s, sys: 799 ms, total: 8.67 s
Wall time: 22min 48s


In [ ]:
preds = clf_pipelineModel.transform(testData)


In [ ]:
preds.select('sentiment','review',"class.result").show(10, truncate=80)


+---------+--------------------------------------------------------------------------------+----------+
|sentiment|                                                                          review|    result|
+---------+--------------------------------------------------------------------------------+----------+
| negative|!!!! MILD SPOILERS !!!!<br /><br />With the exception of THE TERMINATOR and T...|[negative]|
| positive|" Now in India's sunny 'clime, where I use to spend my time as a soldier in t...|[positive]|
| positive|"2001: A Space Odyssey" is set in 2001 and the main character is HAL. A compu...|[negative]|
| negative|"A Christmas Story" is one of many people's all-time most beloved films. ACS ...|[negative]|
| positive|"A Fare to Remember" is a totally derivative, almost ridiculous movie, but ha...|[positive]|
| negative|"A Guy Thing" tries to capture the feeling of "There's Something About Mary" ...|[negative]|
| negative|"A Texas community is beset with a rash of mysterious

In [ ]:
# 9.2

preds_df = preds.select('sentiment','review',"class.result").toPandas()

# The result is an array since in Spark NLP you can have multiple sentences.
# Let's explode the array and get the item(s) inside of result column out

# 9.3
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

In [ ]:
# 9.4 We are going to use sklearn to evalute 
#      the results on test dataset:

from sklearn.metrics import classification_report

# 9.5
print (classification_report(preds_df['result'], preds_df['sentiment']))

              precision    recall  f1-score   support

    negative       0.81      0.79      0.80      5067
    positive       0.79      0.81      0.80      4878

    accuracy                           0.80      9945
   macro avg       0.80      0.80      0.80      9945
weighted avg       0.80      0.80      0.80      9945

